## Homoscedasticity
The basic concern while building/training machine learning model is to obtain a bias free and accurate model across the board. Otherwise, the result from ML model is not worthy for decision making. This is what Homoscedasticity aims for. 

In simple term, homoscedasticity means that the errors (or mistakes) in ML model predictions are evenly spread out, no matter what value we’re predicting i.e. we want to make sure that our prediction errors are consistent. If some errors are spread out unevenly, that's a sign that something is wrong with our model. Thus, the residuals should have constant variance at every level of the independent variable(s).

For example: We collected data from students on the number of hours they studied and their corresponding test scores. When we plot the residuals (the differences between actual scores and predicted scores) against the number of hours studied, we can see that the spread of these residuals stays roughly the same across all hours.

For instance, whether students studied 1 hour or 10 hours, the errors in predicting their scores (how far off our predictions are) are about the same—some are a little high, some a little low, but all clustered evenly around zero.

This even means our model is performing well across all levels of study time, and you can confidently interpret how study hours affect test scores without worrying about changing error sizes.

**Why it is important?**
*  If residuals are no homoscedastic i.e. thye have non-consistent variance, Standard errors may be underestimated or overestimated, leading to invalid conclusions about the significance of predictors.
* If the variance of residuals changes at different levels of the independent variable(s), it complicates the interpretation of the coefficients. This gives the false impression about the importance of certain factors, which can lead to misleading conclusions.
* Non-constant variance can indicate that the model is misspecified. For example, it may suggest that a non-linear relationship exists or that an important variable has been omitted.


**How to Detect Homoscedasticity Issues**
* Plot the residuals against fitted values or independent variables. This is known as residual plot. A funnel shape or pattern indicates heteroscedasticity.
* Perform statistical tests like Breusch-Pagan or White's test which can formally assess homoscedasticity.

**What to Do If Heteroscedasticity Is Detected?**
* Applying transformations (e.g., log, square root) to the dependent variable can sometimes stabilize variance.
* Use Weighted Least Squares technique, which assigns weights to different observations based on the variance of their residuals.
* Using robust standard errors can provide valid inference even in the presence of heteroscedasticity.